In [2]:
import requests
import csv
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from time import sleep
import re
import random
import pandas as pd

In [3]:
headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br", 
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8,sl;q=0.7,mk;q=0.6", 
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36", 
    "Cache-control": "no-cache",
    "Origin": "https://www.avto.net",
    "Referer": "https://www.avto.net/",
    "Content-type": "application/x-www-form-urlencoded",
    "Sec-ch-ua-platform": "Linux",    
}

In [7]:
def get_url(page_number):
    return f"https://www.avto.net/Ads/results.asp?znamka=&model=&modelID=&tip=&znamka2=&model2=&tip2=&znamka3=&model3=&tip3=&cenaMin=0&cenaMax=999999&letnikMin=0&letnikMax=2090&bencin=0&starost2=999&oblika=0&ccmMin=0&ccmMax=99999&mocMin=0&mocMax=999999&kmMin=0&kmMax=9999999&kwMin=0&kwMax=999&motortakt=0&motorvalji=0&lokacija=0&sirina=0&dolzina=&dolzinaMIN=0&dolzinaMAX=100&nosilnostMIN=0&nosilnostMAX=999999&lezisc=&presek=0&premer=0&col=0&vijakov=0&EToznaka=0&vozilo=&airbag=&barva=&barvaint=&EQ1=1000000000&EQ2=1000000000&EQ3=1000000000&EQ4=100000000&EQ5=1000000000&EQ6=1000000000&EQ7=1000100110&EQ8=1010000001&EQ9=1000000000&KAT=1010000000&PIA=&PIAzero=&PSLO=&akcija=0&paketgarancije=&broker=0&prikazkategorije=0&kategorija=0&ONLvid=0&ONLnak=0&zaloga=10&arhiv=0&presort=3&tipsort=DESC&stran={page_number}" 

In [81]:
#with open("response.html", "w") as f:
#    f.write(r.text)
# with open("response.html", "r") as f:
#     soup = BeautifulSoup(f.read(), "html.parser")
#     soups.append(soup)

In [5]:
regex = re.compile('.*Logo.*')

def extract_car_data(car):    
    title = car.find("div", class_="GO-Results-Naziv").text.strip().replace(",", "")
    try:
        broker = car.find('div', {'class': regex}).a['href']
    except:
        broker = ""    
    year = car.find('tbody').find('tr').findAll()[1].text
    car_data = [title, broker, year]    
    return car_data

In [13]:
# columns = ['title', 'broker', 'year']
# with open('data.csv', 'w') as file:
#     file.write(", ".join(columns))
#     file.write("\n")

In [8]:
df1 = pd.read_csv("data2.csv")
df2 = pd.read_csv("data3.csv")

In [13]:
df = pd.concat([df1, df2], ignore_index=True)
df.columns = ['title', 'broker', 'year']

In [16]:
df_temp = df['title'].str.split(expand=True, n=2)
df_temp.columns = ['brand', 'model', 'info']

In [18]:
df = pd.concat([df_temp, df[['year', 'broker']]], axis=1)

In [21]:
def get_broker_id(broker_base_url):
    return broker_base_url.split('?')[1].split('&')[0][7:]

In [22]:
df['broker_id'] = df['broker'].apply(get_broker_id)

In [23]:
df.head()

,brand,model,info,year,broker,broker_id
0,Kia,Sportage,2WD 1.7 CRDi Urban 1LAST. SLO.NA OBROKE-BREZ P...,2015,results321_makes.asp?broker=14095&oglasrubrik...,14095
1,Mercedes-Benz,E-Razred,E 220 d 4MATIC+LED+PANORAMA+WIDESCREEN+VL.KLJUKA,2017,results321_makes.asp?broker=14947&oglasrubrik...,14947
2,BMW,M4,PERLA-CARBON-KAMERA-MEMORY,2014,results321_makes.asp?broker=14775&oglasrubrik...,14775
3,Porsche,Cayenne,4.8 b+USNJE+MODIFICIRAN+F1+ALU21+LEASING+TOP,2008,results321_makes.asp?broker=16351&oglasrubrik...,16351
4,BMW,serija,5 Touring: 530d xDrive -84.000km-TV-FULL LED-P...,2017,results321_makes.asp?broker=15695&oglasrubrik...,15695


In [27]:
def year_category(year):
    if year < 2006:
        return "old"
    elif 2006 <= year <= 2012:
        return "interest"
    else:
        return "new"        

In [28]:
def get_broker_url(base_url):
    return f"https://www.avto.net/Ads/{base_url}"

In [30]:
df['year_category'] = df['year'].apply(year_category)

In [34]:
df['year_category'].value_counts()

interest    4475
new         2651
old          994
Name: year_category, dtype: int64

In [37]:
german_brands = ['Mercedes-Benz', 'Volkswagen', 'BMW', 'Audi']

In [40]:
df_filter = df[(df['brand'].isin(german_brands)) & (df['year_category']=='interest')]

In [41]:
df_filter.head()

,brand,model,info,year,broker,broker_id,year_category
11,Volkswagen,Passat,2.0 TDI Comfortline DPF,2010,results321_makes.asp?broker=10074&oglasrubrik...,10074,interest
21,Mercedes-Benz,A-Razred,A 160 CDI BLUEEFF ELEGANCE,2011,results321_makes.asp?broker=11509&oglasrubrik...,11509,interest
28,Audi,A6,Allroad 3.0 TDI quattro tiptronic,2007,results321_makes.asp?broker=14359&oglasrubrik...,14359,interest
61,Volkswagen,Touran,2.0 TDI-7 SEDEŽEV-TEMPOMAT-PDC-16COL,2011,results321_makes.asp?broker=15934&oglasrubrik...,15934,interest
64,Volkswagen,Golf,1.9 TDI Comfortline-SLO-KLIMA-5 VRAT-ODLIČEN,2006,results321_makes.asp?broker=14489&oglasrubrik...,14489,interest


In [56]:
df_top = df_filter.groupby('broker_id').size().reset_index(name='counts').sort_values('counts', ascending=False).head()

In [57]:
df_top

,broker_id,counts
28,14634,104
17,13781,101
54,18002,80
23,14446,65
31,15096,37


In [67]:
df_filter[df_filter['broker_id'] == '14634'].groupby(['brand', 'model']).size().reset_index(name='counts')

,brand,model,counts
0,Mercedes-Benz,B-Razred,79
1,Volkswagen,Touran,25


In [68]:
df_filter[df_filter['broker_id'] == '14634']

,brand,model,info,year,broker,broker_id,year_category
957,Volkswagen,Touran,1.6 TDI - 6 PRESTAV - KLIMA - PDC - SLO -TEMPOMAT,2011,results321_makes.asp?broker=14634&oglasrubrik...,14634,interest
965,Mercedes-Benz,B-Razred,B 180 CDI- 6 PRESTAV - KLIMA - ALU - GR. SEDEŽEV,2012,results321_makes.asp?broker=14634&oglasrubrik...,14634,interest
1005,Mercedes-Benz,B-Razred,B 180 CDI- 6 PRESTAV - KLIMA - ALU - GR. SEDEŽEV,2012,results321_makes.asp?broker=14634&oglasrubrik...,14634,interest
1045,Mercedes-Benz,B-Razred,B 180 CDI- 6 PRESTAV - KLIMA - ALU - GR. SEDEŽEV,2012,results321_makes.asp?broker=14634&oglasrubrik...,14634,interest
1085,Mercedes-Benz,B-Razred,B 180 CDI- 6 PRESTAV - KLIMA - ALU - GR. SEDEŽEV,2012,results321_makes.asp?broker=14634&oglasrubrik...,14634,interest
...,...,...,...,...,...,...,...
4011,Mercedes-Benz,B-Razred,B 180 CDI- 6 PRESTAV - KLIMA - ALU - GR. SEDEŽEV,2012,results321_makes.asp?broker=14634&oglasrubrik...,14634,interest
4043,Volkswagen,Touran,1.6 TDI - 6 PRESTAV - KLIMA - PDC - SLO -TEMPOMAT,2011,results321_makes.asp?broker=14634&oglasrubrik...,14634,interest
4051,Mercedes-Benz,B-Razred,B 180 CDI- 6 PRESTAV - KLIMA - ALU - GR. SEDEŽEV,2012,results321_makes.asp?broker=14634&oglasrubrik...,14634,interest
4083,Volkswagen,Touran,1.6 TDI - 6 PRESTAV - KLIMA - PDC - SLO -TEMPOMAT,2011,results321_makes.asp?broker=14634&oglasrubrik...,14634,interest


In [6]:
columns = ['title', 'broker', 'year']
with open('data3.csv', 'w') as file:
    file.write(",".join(columns))
    file.write("\n")

In [70]:
def scraper(page_from, page_to):
    for page_number in range(page_from, page_to):
        url = get_url(page_number)
        r = requests.get(url, headers=headers)
        soup = BeautifulSoup(r.content, "html.parser")
        with open('data3.csv', 'a') as file:
            for car in soup.findAll('div', class_='GO-Results-Row'):
                car_data = extract_car_data(car)
                file.write(", ".join(car_data))
                file.write("\n")
        print(page_number)
        sleep(random.randint(10,20))

In [69]:
len(df)

8120